In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from time import time
from utils import train_neural_network, load_warwick

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [14]:
batch_size = 16
X_train, Y_train, X_test, Y_test = load_warwick(device)
# create dataloader
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# look at the size of image and label
print(X_train.size())
print(Y_train.size())

torch.Size([85, 3, 128, 128])
torch.Size([85, 128, 128])


In [16]:
# Define the modified CNN architecture
class ModifiedCNN(nn.Module):
    def __init__(self):
        super(ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Add padding to preserve spatial dimensions
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # Add padding to preserve spatial dimensions
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)  # Add padding to preserve spatial dimensions
        self.relu3 = nn.ReLU()
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)  # Add padding to preserve spatial dimensions
        self.relu4 = nn.ReLU()
        self.conv5 = nn.Conv2d(128, 1, 3, padding=1)  # Adjust output channels to 1
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        return x.squeeze()

# Create an instance of the modified CNN
model = ModifiedCNN()

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
def dice_coefficient(outputs, targets):
    eps = 1e-8
    intersection = torch.sum(abs(outputs) * abs(targets))
    union = torch.sum(outputs) + torch.sum(targets)
    dice = (2. * intersection) / abs(union + eps)
    return dice

In [18]:
def training_curve_plot(title, train_losses, test_losses, train_accuracy, test_accuracy):
    """ 
    convenience function for plotting train and test loss and accuracy
    """
    lg=13
    md=8
    sm=7
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    fig.suptitle(title, fontsize=md)
    x = range(1, len(train_losses)+1)
    axs[0].plot(x, train_losses, label=f'Final train loss: {train_losses[-1]:.4f}')
    axs[0].plot(x, test_losses, label=f'Final test loss: {test_losses[-1]:.4f}')
    axs[0].set_title('Losses', fontsize=md)
    axs[0].set_xlabel('Iteration', fontsize=md)
    axs[0].set_ylabel('Loss', fontsize=md)
    axs[0].legend(fontsize=sm)
    axs[0].tick_params(axis='both', labelsize=sm)
    # Optionally use a logarithmic y-scale
    #axs[0].set_yscale('log')
    axs[0].grid(True, which="both", linestyle='--', linewidth=0.5)
    axs[1].plot(x, train_accuracy, label=f'Final train dice: {train_accuracy[-1]:.4f}%')
    axs[1].plot(x, test_accuracy, label=f'Final test dice: {test_accuracy[-1]:.4f}%')
    axs[1].set_title('dice', fontsize=md)
    axs[1].set_xlabel('Iteration', fontsize=md)
    axs[1].set_ylabel('dice (%)', fontsize=sm)
    axs[1].legend(fontsize=sm)
    axs[1].tick_params(axis='both', labelsize=sm)
    axs[1].grid(True, which="both", linestyle='--', linewidth=0.5)
    plt.show()

In [24]:
def train_network(model: nn.Module, criteria: nn.Module, 
                  optimizer: torch.optim.Optimizer, num_epochs: int, train_loader: DataLoader, 
                  test_loader: DataLoader, device: torch.device, model_name: str, lr: float) -> None:
    """
    Train a neural network model
    
    Args:
        model: The neural network model to be trained
        criteria: The loss function
        optimizer: The optimizer
        num_epochs: The number of epochs to train the model
        train_loader: The training data loader
        test_loader: The test data loader
        device: The device to run the model on
    """
    train_losses = []
    test_losses = []
    train_dice_scores = []
    test_dice_scores = []
    start = time()
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_dice = 0

        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criteria(outputs, labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

            # Calculate Dice score
            outputs = (outputs > 0.5).float()
            dice = dice_coefficient(outputs, labels)
            train_dice += dice.item()
        
        train_losses.append(train_loss / len(train_loader))
        train_dice_scores.append(train_dice / len(train_loader))
    
        model.eval()
        test_loss = 0
        test_dice = 0

        with torch.no_grad():
            for i, (images, labels) in enumerate(test_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criteria(outputs, labels)
                test_loss += loss.item()

                # Calculate Dice score
                outputs = (outputs > 0.5).float()
                dice = dice_coefficient(outputs, labels)
                test_dice += dice.item()
            
            test_losses.append(test_loss / len(test_loader))
            test_dice_scores.append(test_dice / len(test_loader))
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}, Train Dice Score: {train_dice_scores[-1]:.4f}, Test Dice Score: {test_dice_scores[-1]:.4f}')
    total = time() - start
    plot_title = f'{model_name} - lr: {lr} - Total Time: {total//60:.0f}min {total%60:.2f}s, Epochs: {num_epochs}'
    training_curve_plot(plot_title,train_losses, test_losses, train_dice_scores, test_dice_scores)

In [20]:
print(f"Input shape: {train_loader.dataset.tensors[0].shape}")
print(f"Label shape: {train_loader.dataset.tensors[1].shape}")

Input shape: torch.Size([85, 3, 128, 128])
Label shape: torch.Size([85, 128, 128])


In [25]:
# Hyperparameters
num_epochs = 100
learning_rate = 0.001   
model = ModifiedCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_network(model, criterion, optimizer, num_epochs=num_epochs,train_loader=train_loader, test_loader=test_loader,  device=device, model_name='Modified CNN', lr=learning_rate)

Epoch 1/100, Train Loss: 1.0357, Test Loss: 0.7105, Train Dice Score: 0.1490, Test Dice Score: 0.3827
Epoch 2/100, Train Loss: 0.6918, Test Loss: 0.6923, Train Dice Score: 0.2526, Test Dice Score: 0.0525
Epoch 3/100, Train Loss: 0.6800, Test Loss: 0.6724, Train Dice Score: 0.0118, Test Dice Score: 0.0188
Epoch 4/100, Train Loss: 0.6579, Test Loss: 0.6617, Train Dice Score: 0.1289, Test Dice Score: 0.3773
Epoch 5/100, Train Loss: 0.6210, Test Loss: 0.6981, Train Dice Score: 0.5156, Test Dice Score: 0.4885
Epoch 6/100, Train Loss: 0.5909, Test Loss: 0.6895, Train Dice Score: 0.6034, Test Dice Score: 0.5007
Epoch 7/100, Train Loss: 0.5706, Test Loss: 0.6759, Train Dice Score: 0.5895, Test Dice Score: 0.5291
Epoch 8/100, Train Loss: 0.5666, Test Loss: 0.7175, Train Dice Score: 0.5717, Test Dice Score: 0.5673
Epoch 9/100, Train Loss: 0.5578, Test Loss: 0.6894, Train Dice Score: 0.5979, Test Dice Score: 0.5658
Epoch 10/100, Train Loss: 0.5472, Test Loss: 0.6329, Train Dice Score: 0.6221, Tes

In [22]:
test_model = ModifiedCNN().to(device)
optimizer = torch.optim.Adam(test_model.parameters(), lr=0.001)
test_model.train()
optimizer.zero_grad()
outputs = test_model(X_train[0])
loss = criterion(outputs, Y_train[0])
train_loss = loss
loss.backward()
optimizer.step()